In [ ]:
!nvidia-smi
import os
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive' 
os.chdir(path)
os.listdir(path)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

#残差网络

In [11]:
class ResidualBlock(nn.Module):
  def __init__(self,channels):
    super(ResidualBlock,self).__init__()
    self.channels = channels
    self.conv1 = nn.Conv2d(channels,channels,kernel_size=3,padding=1)
    self.conv2 = nn.Conv2d(channels,channels,kernel_size=3,padding=1)
  def forward(self,x):
    y=F.relu(self.conv1(x))
    y=self.conv2(y)
    return F.relu(x+y)

#整个网络

In [12]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.conv1 = nn.Conv2d(1,16,kernel_size=5)
    self.conv2 = nn.Conv2d(16,32,kernel_size=5)
    
    self.mp = nn.MaxPool2d(2)

    self.rblock1 = ResidualBlock(16)
    self.rblock2 = ResidualBlock(32)

    self.fc = nn.Linear(512,10)
  def forward(self,x):
    batch_size=x.size(0)
    #(batch_size,1,28,28)
    x = F.relu(self.conv1(x))#(b,16,24,24)
    x = self.mp(x)#(b,16,12,12)
    x = self.rblock1(x)#(b,16,12,12)

    x = F.relu(self.conv2(x))#(b,32,8,8)
    x = self.mp(x)#(b,32,4,4)
    x = self.rblock2(x)#(b,32,4,4)

    
    x = x.view(batch_size,-1)#(b,512)
    x = self.fc(x)
    return x

#配置

In [16]:
batch_size = 64
transform = transforms.Compose([
  transforms.ToTensor(),          
  transforms.Normalize((0.1307,),(0.3081,))                
])

train_set = datasets.MNIST(root='Colab Notebooks/dataset',
              train=True,           
              transform=transform,       
              download=True)
test_set = datasets.MNIST(root='Colab Notebooks/dataset',
              train=False,          
              transform=transform,
              download=True)

#加载数据集
train_loader = DataLoader(dataset=train_set,
              batch_size=batch_size,
              shuffle=True)
test_loader = DataLoader(dataset=test_set,
              batch_size=batch_size,
              shuffle=False)

model = Model()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

#使用GPU计算
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

Model(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (rblock1): ResidualBlock(
    (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (rblock2): ResidualBlock(
    (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (fc): Linear(in_features=512, out_features=10, bias=True)
)

#训练

In [15]:
def train(epoch):
  running_loss = 0.0
  for batch_idx,(input,target) in enumerate(train_loader):
    input,target = input.to(device), target.to(device)#使用GPU计算
    optimizer.zero_grad()
    output = model(input)
    loss = criterion(output,target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 300 == 299:
      print('[%d,%5d] loss: %.3f' % (epoch + 1, batch_idx +1, running_loss/300))
      running_loss = 0.0

def test():
  correct = 0
  total = 0
  with torch.no_grad(): 
    for input,target in test_loader:
      input,target = input.to(device), target.to(device)#使用GPU计算
      output = model(input)
      _,predicted = torch.max(output.data,dim=1)
      total+=target.size(0)
      correct += (predicted == target).sum().item()
  print('Accuracy on test set:%d %%' % (100*correct/total))


for epoch in range(10):
  train(epoch)
  test()

[1,  300] loss: 0.553
[1,  600] loss: 0.136
[1,  900] loss: 0.103
Accuracy on test set:97 %
[2,  300] loss: 0.087
[2,  600] loss: 0.072
[2,  900] loss: 0.069
Accuracy on test set:98 %
[3,  300] loss: 0.058
[3,  600] loss: 0.060
[3,  900] loss: 0.052
Accuracy on test set:98 %
[4,  300] loss: 0.046
[4,  600] loss: 0.049
[4,  900] loss: 0.043
Accuracy on test set:98 %
[5,  300] loss: 0.037
[5,  600] loss: 0.041
[5,  900] loss: 0.038
Accuracy on test set:98 %
[6,  300] loss: 0.035
[6,  600] loss: 0.033
[6,  900] loss: 0.032
Accuracy on test set:98 %
[7,  300] loss: 0.030
[7,  600] loss: 0.031
[7,  900] loss: 0.029
Accuracy on test set:98 %
[8,  300] loss: 0.025
[8,  600] loss: 0.027
[8,  900] loss: 0.027
Accuracy on test set:98 %
[9,  300] loss: 0.026
[9,  600] loss: 0.026
[9,  900] loss: 0.021
Accuracy on test set:98 %
[10,  300] loss: 0.021
[10,  600] loss: 0.019
[10,  900] loss: 0.024
Accuracy on test set:98 %
